# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [37]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [38]:
# your code here

orders = pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [44]:
# your code here
#Sub Problem 1: How to aggregate the amount_spent for unique customers?

customer_amount = orders.groupby("CustomerID")["amount_spent"].sum().sort_values()

customer_amount = pd.DataFrame(customer_amount)

# Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

quantile_range = orders.groupby("CustomerID")["amount_spent"].sum().quantile([0, 0.25, 0.5, 0.75, 0.95, 1])

# Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

VIP = ordersgroup[ordersgroup["amount_spent"] > 5840.182]

condition = (customer_amount["amount_spent"] > 1661.640) & (customer_amount["amount_spent"] < 5840.182)

preferred = customer_amount[condition]

preferred

,amount_spent
CustomerID,
13012,1661.84
12530,1662.28
12912,1662.30
16115,1667.97
17656,1674.69
...,...
13178,5725.47
17686,5739.46
15218,5756.89


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [50]:
# your code here

country_groups = orders.groupby(["Country", "CustomerID"])["amount_spent"].sum()

country_groups = pd.DataFrame(country_groups)

country_VIP= country_groups[country_groups["amount_spent"] > 5840.182]

country_VIP.groupby(["Country"]).count().sort_values(by = "amount_spent", ascending=False).iloc[0:5]

# The country with more VIP customers is UK.

,amount_spent
Country,
United Kingdom,177
Germany,10
France,9
Switzerland,3
Spain,2


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [49]:
# your code here

VIP_Preferred = country_groups[country_groups["amount_spent"] > 1661.640]

country_groups.groupby(["Country"]).count().sort_values(by = "amount_spent", ascending = False).iloc[0:5]

# The country with more VIP and Preferred customers combined is UK.

,amount_spent
Country,
United Kingdom,3921
Germany,94
France,87
Spain,30
Belgium,25
